In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import lib.globals as globals
import os 


In [ ]:
dataset_path = os.path.join(globals.DATA_FOLDER,'training_set.json')

In [ ]:
import lib.data_handling as handling

squad_dataset = handling.RawSquadDataset(dataset_path)

In [ ]:
rnn_manager = handling.RecurrentDataManager(squad_dataset,False)

In [ ]:
rnn_manager.raw_dataset.df.loc[5,'answer']

In [ ]:
print(rnn_manager.hf_dataset[4:6])

In [ ]:
import lib.utils as utils 

model, vocab = utils.load_embedding_model()

In [ ]:
model.key_to_index == vocab

In [ ]:
model.allocate_vecattrs()
glove_model_path = os.path.join(globals.DATA_FOLDER, "glove_vectors.txt") 
model.save_word2vec_format(glove_model_path,binary=True)

In [ ]:
model.get_vector('[PAD]')

In [ ]:
'[PAD]' in model

In [ ]:
for k,v in vocab.items():
    if v == 400001:
        print(k)

In [ ]:
from tokenizers import  Tokenizer
from tokenizers.models import WordLevel
from tokenizers.normalizers import Lowercase, Sequence, Strip, StripAccents
from tokenizers.pre_tokenizers import Punctuation
from tokenizers.pre_tokenizers import Sequence as PreSequence
from tokenizers.pre_tokenizers import Whitespace

In [ ]:
from datasets import Dataset 

hf_dataset = Dataset.from_pandas(squad_dataset.df)

In [ ]:
hf_dataset

In [ ]:
# tokenizer.enable_padding(
#         direction="right", pad_id=vocab['[PAD]'], pad_type_id=1, pad_token='[PAD]'
#     )

def convert_to_features(batch):
    #print(example_batch['context'][0])
    encodings = tokenizer.encode_batch(batch['context'])
    # target_encodings = tokenizer.encode(example_batch['target_text'], pad_to_max_length=True, max_length=16)

    encodings = {
        'input_ids': [e.ids for e in encodings], 
        'attention_mask': [e.attention_mask for e in encodings],
        'offset':[e.offsets for e in encodings]
    }

    return encodings



hf_dataset2 = hf_dataset.map(convert_to_features,batched=True)



# hf_dataset.set_transform(pad_batch,output_all_columns=False)
# 'label_token' : [list(e.char_to_token(e.label_char[0],3),e.char_to_token(e.label_char[0],3)) for e in encodings]

In [ ]:
hf_dataset

In [ ]:
import torch

In [ ]:
tokenizer = Tokenizer(WordLevel(vocab,unk_token=globals.UNK_TOKEN))
tokenizer.normalizer = Sequence([StripAccents(), Lowercase(), Strip()])
tokenizer.pre_tokenizer = PreSequence([Whitespace(), Punctuation()])
tokenizer.enable_padding(direction="right", pad_id=vocab[globals.PAD_TOKEN], pad_type_id=1, pad_token=globals.PAD_TOKEN)

In [ ]:
from tokenizers import Encoding

def transform(batch):

    context_encodings: list[Encoding] = tokenizer.encode_batch(batch['context'])
    question_encodings: list[Encoding] = tokenizer.encode_batch(batch['question'])

    starts = list(map(lambda x: x[0],batch['label_char']))
    ends = list(map(lambda x: x[1],batch['label_char']))

    encodings = {
        'context_ids': [e.ids for e in context_encodings],
        'question_ids': [e.ids for e in question_encodings],
        'context_mask': [e.attention_mask for e in context_encodings],
        'question_mask': [e.attention_mask for e in question_encodings],
        'label_token_start': [e.char_to_token(starts[i]) for i,e in enumerate(context_encodings)],
        'label_token_end': [e.char_to_token(ends[i]-1) for i,e in enumerate(context_encodings)]        
    }

    return encodings

hf_dataset.set_transform(transform,output_all_columns=False)

In [ ]:
tokenizer.padding

In [ ]:
squad_dataset.df.loc[4,'question']
squad_dataset.df.loc[4,'answer']

In [ ]:
print(hf_dataset[4:6])

In [ ]:
sa = hf_dataset[4]['label_token_start']
ta = hf_dataset[4]['context_ids'][sa]
se = hf_dataset[4]['label_token_end']
te = hf_dataset[4]['context_ids'][se]
tokenizer.decode([ta,te])



In [ ]:
# 
len(hf_dataset[6]['padded_ids']) - len(hf_dataset[0]['padded_ids'])

In [ ]:
print(tokenizer.padding)

In [ ]:
print(len(hf_dataset[56459]['input_ids']))

In [ ]:
print(tokenizer.id_to_token(1962))
print(tokenizer.token_to_id('[PAD]'))

In [ ]:
hf_dataset[0,1]